<a href="https://colab.research.google.com/github/sooyun1202/NLP/blob/main/BERT%EB%AA%A8%EB%8D%B8_%EC%98%81%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet # package installer for python

     |████████████████████████████████| 1.5MB 14.7MB/s 
     |████████████████████████████████| 2.9MB 57.6MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 


In [2]:
import torch
from transformers import BertModel, BertTokenizer

In [3]:
pretrained_weights = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
model = BertModel.from_pretrained(pretrained_weights)

데이터셋

In [9]:
import json

data = {'train': {'speaker': [], 'utterance': [], 'emotion': []},
        'dev': {'speaker': [], 'utterance': [], 'emotion': []},
        'test': {'speaker': [], 'utterance': [], 'emotion': []}}

for dtype in ['train', 'dev', 'test']:
  for dialog in json.loads(open('friends_' + dtype + '.json').read()):
    for line in dialog:
      data[dtype]['speaker'].append(line['speaker'])
      data[dtype]['utterance'].append(line['utterance'])
      data[dtype]['emotion'].append(line['emotion'])

In [10]:
e2i_dict = dict((emo, i) for i, emo in enumerate(set(data['train']['emotion'])))
i2e_dict = {i: e for e, i in e2i_dict.items()}

In [8]:
#dev데이터 개수
with open('friends_train.json') as json_dev_file:
    json_dev_data = json.load(json_dev_file)

devCnt = 0

for i in range(len(json_dev_data)):
  for j in range(len(json_dev_data[i])):
    devCnt = devCnt+1

print('dev데이터 개수 :',devCnt,'개')

#train데이터 개수
with open('friends_train.json') as json_train_file:
    json_train_data = json.load(json_train_file)

trCnt = 0

for i in range(len(json_train_data)):
  for j in range(len(json_train_data[i])):
    trCnt = trCnt+1

print('train데이터 개수 :',trCnt,'개')

#test데이터 개수
with open('friends_test.json') as json_test_file:
    json_test_data = json.load(json_test_file)

teCnt = 0

for i in range(len(json_test_data)):
  for j in range(len(json_test_data[i])):
    teCnt = teCnt+1

print('test데이터 개수 :',teCnt,'개')

train데이터 개수 : 10561 개
test데이터 개수 : 2764 개
dev데이터 개수 : 10561 개


In [11]:
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert_tokenizer = BertTokenizer.from_pretrained(pretrained_weights)
    self.bert_model = BertModel.from_pretrained(pretrained_weights)
    self.linear = torch.nn.Linear(768, len(e2i_dict))

  def forward(self, utterance):
    tokens = self.bert_tokenizer.tokenize(utterance)
    tokens = ['[CLS]'] + tokens + ['[SEP]'] # (len)
    ids = [tokenizer.convert_tokens_to_ids(tokens)] # (bat=1, len)
    input_tensor = torch.tensor(ids).cuda()

    hidden_tensor = self.bert_model(input_tensor)[0] # (bat, len, hid)
    hidden_tensor = hidden_tensor[:, 0, :] # (bat, hid)
    logit = self.linear(hidden_tensor)
    return logit

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(true_list, pred_list):
  precision = precision_score(true_list, pred_list, average=None)
  recall = recall_score(true_list, pred_list, average=None)
  micro_f1 = f1_score(true_list, pred_list, average='micro')
  print('precision:\t', ['%.4f' % v for v in precision])
  print('recall:\t\t', ['%.4f' % v for v in recall])
  print('micro_f1: %.6f' % micro_f1)

In [13]:
pretrained_weights = 'bert-base-uncased'
learning_rate = 2e-5 
n_epoch = 3

In [14]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import torch
from tqdm import tqdm_notebook

model = Model()
model.cuda()
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), learning_rate) #Adam, Adagrad, SGD

for i_epoch in range(n_epoch):
  print('i_epoch:', i_epoch)

  model.train()
  for i_batch in tqdm_notebook(range(len(data['train']['utterance']))):
    logit = model(data['train']['utterance'][i_batch])
    target = torch.tensor([e2i_dict[data['train']['emotion'][i_batch]]]).cuda()
    loss = criterion(logit, target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  
  model.eval()
  pred_list, true_list = [], []
  for i_batch in tqdm_notebook(range(len(data['dev']['utterance']))):
    logit = model(data['dev']['utterance'][i_batch])
    _, max_idx = torch.max(logit, dim=-1)
    pred_list += max_idx.tolist()
    true_list += [e2i_dict[data['dev']['emotion'][i_batch]]]
  evaluate(pred_list, true_list) # print results

i_epoch: 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



precision:	 ['0.2097', '0.0000', '0.3176', '0.8941', '0.1449', '0.5610', '0.1739', '0.5762']
recall:		 ['0.4815', '0.0000', '0.3913', '0.6298', '0.3563', '0.5188', '0.4000', '0.5613']
micro_f1: 0.568761
i_epoch: 1


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



precision:	 ['0.3387', '0.0000', '0.2824', '0.8147', '0.2196', '0.5366', '0.2609', '0.5629']
recall:		 ['0.3333', '0.0000', '0.4444', '0.6515', '0.3113', '0.5238', '0.3750', '0.5519']
micro_f1: 0.550934
i_epoch: 2



precision:	 ['0.3065', '0.0345', '0.3294', '0.7984', '0.2009', '0.4797', '0.3043', '0.6159']
recall:		 ['0.4222', '0.2000', '0.3457', '0.6689', '0.3007', '0.5842', '0.1944', '0.5138']
micro_f1: 0.544992


In [16]:
import csv

dialogs = []
dialogs.append([])

with open('en_data.csv', newline='') as csvfile:
  spamreader = csv.reader(csvfile)
  for i,row in enumerate(spamreader):
    if i!=0:
      dialogs[0].append({'id':row[0],'speaker':row[3], 'utterance':row[4]})

from collections import OrderedDict

labeled = []
for dialog in tqdm_notebook(dialogs):
  dialog_list = []
  for line in dialog:
    logit = model(line['utterance'])
    _, max_idx = torch.max(logit, dim=-1)
    pred_emotion = max_idx.tolist()[0]

    line_dict = OrderedDict()
    line_dict['Id'] = line['id']
    line_dict['speaker'] = line['speaker']
    line_dict['utterance'] = line['utterance']
    line_dict['emotion'] = i2e_dict[pred_emotion]
    dialog_list.append(line_dict)
  labeled.append(dialog_list)

with open('labeled.csv', 'w', newline='') as csvfile:
    fieldnames = ['Id', 'Predicted']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in dialog_list:
      writer.writerow({'Id': row['Id'], 'Predicted': row['emotion']})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


참고: https://colab.research.google.com/drive/1EMzEfTYjYLgEHjCCP1vEr9oOZLXMocGh?usp=sharing